In [1]:
#importing libraries
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from collections import Counter
import seaborn as sns
from pymongo import MongoClient

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = MongoClient(conn)

In [21]:
beer_df = pd.read_csv('beer_reviews (1).csv')
beer_df.head(-1)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586608,14359,The Defiant Brewing Company,1162685856,3.5,4.0,4.0,treehugger02010,Pumpkin Ale,3.5,3.0,The Horseman's Ale,5.2,33061
1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061


In [39]:
beer_count=beer_df.groupby(['beer_beerid', 'beer_name']).count()['review_overall']
beer_count

beer_beerid  beer_name                      
3            Cactus Queen IPA                     3
4            Wildcatter's Crude Stout            10
5            Amber                              424
6            Turbodog                           877
7            Purple Haze                        659
                                               ... 
77313        Aass Gourmet Pale Ale                1
77314        Betty Brown Norwegian Brwon Ale      1
77315        Icelandic White Beer                 1
77316        Crank Yanker IPA                     1
77317        Belgo Sutra                          1
Name: review_overall, Length: 66055, dtype: int64

In [40]:
beer_mean=beer_df.groupby(['beer_beerid', 'beer_name']).mean()
beer_mean

,,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
beer_beerid,beer_name,,,,,,,,
3,Cactus Queen IPA,2.0,1.034934e+09,4.166667,4.000000,3.833333,4.166667,4.166667,NaN
4,Wildcatter's Crude Stout,2.0,1.065482e+09,3.700000,3.800000,3.950000,3.700000,3.450000,NaN
5,Amber,3.0,1.192080e+09,3.549528,3.205189,3.485849,3.318396,3.340802,4.5
6,Turbodog,3.0,1.196812e+09,3.706956,3.515964,3.836374,3.513113,3.645382,5.6
7,Purple Haze,3.0,1.201470e+09,3.266313,3.179059,3.251138,3.106980,3.101669,4.2
...,...,...,...,...,...,...,...,...,...
77313,Aass Gourmet Pale Ale,185.0,1.326236e+09,3.000000,3.000000,4.000000,3.000000,3.000000,4.7
77314,Betty Brown Norwegian Brwon Ale,18604.0,1.326237e+09,3.500000,3.500000,4.000000,3.000000,4.000000,4.7
77315,Icelandic White Beer,27839.0,1.326239e+09,3.500000,4.500000,3.000000,3.000000,3.500000,5.2


In [41]:
beer_overall = beer_mean.drop(columns=['brewery_id', 'review_time', 'beer_abv'])
beer_overall.insert('review_count', beer_count['review_overall'])

KeyError: 'review_overall'

In [4]:
#shape of beer data
print("Number of reviews: ", beer_df.shape[0])
print("Number of features: ", beer_df.shape[1])
print("Total number of breweries; ", len(beer_df['brewery_name'].value_counts()))
print("Total number of beer style; ", len(beer_df['beer_style'].value_counts()))
print("Total number of beer names; ", len(beer_df['beer_name'].value_counts()))
print("Total number of brewery names; ", len(beer_df['brewery_name'].value_counts()))
print("Total number of brewery ID; ", len(beer_df['brewery_id'].value_counts()))

Number of reviews:  1586614
Number of features:  13
Total number of breweries;  5742
Total number of beer style;  104
Total number of beer names;  56857
Total number of brewery names;  5742
Total number of brewery ID;  5840


In [14]:
# find top 10 popular Breweries by review counts
popular_brewery = beer_df.groupby('brewery_name').brewery_name.count()
popular_brewery = popular_brewery.sort_values(ascending= False)
popular_breweries = popular_brewery.iloc[0:10]
popular_breweries

brewery_name
Boston Beer Company (Samuel Adams)    39444
Dogfish Head Brewery                  33839
Stone Brewing Co.                     33066
Sierra Nevada Brewing Co.             28751
Bell's Brewery, Inc.                  25191
Rogue Ales                            24083
Founders Brewing Company              20004
Victory Brewing Company               19479
Lagunitas Brewing Company             16837
Avery Brewing Company                 16107
Name: brewery_name, dtype: int64

In [18]:
# top 10 beers with reviews count
top_100000_popular_beers = pd.DataFrame(beer_df.groupby('beer_name').beer_name.count().sort_values(ascending = False).iloc[0:100000])
top_100000_popular_beers.columns = ['Beer Names with Reviews counts']
top_100000_popular_beers

,Beer Names with Reviews counts
beer_name,
90 Minute IPA,3290
India Pale Ale,3130
Old Rasputin Russian Imperial Stout,3111
Sierra Nevada Celebration Ale,3000
Two Hearted Ale,2728
...,...
Krug-Bräu Pilsner,1
Krug-Bräu Festbier,1
Krug Premium,1


In [16]:
# top 10 beers with highest ratings
Top_beer_ratings = beer_df[['beer_name','review_overall']].groupby('beer_name').review_overall.agg('mean').sort_values(ascending = False)
Top_beer_ratings = Top_beer_ratings.iloc[0:10]
Top_beer_ratings.reset_index()

,beer_name,review_overall
0,Taeberry Boch,5.0
1,Wasatch Irish Stout,5.0
2,"Pale Ale S.C.A.G. (Simcoe, Columbus, Amarillo ...",5.0
3,Distorter Porter,5.0
4,Louwaege's Stout,5.0
5,Belgian Country Pale Ale,5.0
6,Cappy's Cherry Wheat,5.0
7,C. Brown's Pumpkin Ale,5.0
8,John's Mild Bitter,5.0
9,Raspberry Sparkle,5.0


In [17]:
# missing data by rows
beer_row_clean = beer_df.dropna(subset=['brewery_id','brewery_name', 'beer_name', 'review_taste', 'review_overall', 'review_palate', 'review_aroma','beer_style', 'review_appearance'], how='any')
beer_row_clean.head(100000)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2958,BFM Brasserie Des Franches-Montagnes,1289096705,2.0,3.0,3.0,caseyf,Bière de Garde,2.5,3.0,Abbaye De Saint Bon-Chien Grand Cru (Aged In M...,11.0,58679
99996,2958,BFM Brasserie Des Franches-Montagnes,1274886334,3.0,3.5,3.5,Anthony1,Bière de Garde,3.0,3.0,Abbaye De Saint Bon-Chien Grand Cru (Aged In M...,11.0,58679
99997,2958,BFM Brasserie Des Franches-Montagnes,1210546150,2.5,2.5,4.0,ChainGangGuy,Belgian Strong Pale Ale,3.0,2.5,Cuvée Du 8ème,7.5,39654
99998,2958,BFM Brasserie Des Franches-Montagnes,1195557936,4.0,4.0,3.5,Bov,Belgian Strong Pale Ale,4.0,4.0,Cuvée Du 6ème,8.0,39655


In [20]:
beer_df = beer_row_clean.head(100000).loc[beer_row_clean['beer_beerid']==top_100000_popular_beers.index]

ValueError: ('Lengths must match to compare', (1586599,), (56857,))

In [8]:
# Main Data Tables

# brewery_df will have all the brewery information, specifically the id, and the brewery name.
brewery_df = beer_df[['brewery_id','brewery_name', 'beer_beerid', 'beer_name']].drop_duplicates(keep='first')
brewery_df1 = brewery_df.head(100000)
# convert brewery_df to the list of doctionary inorder to import them to MongoDB
table_1 = []

# for brewery in beer_df:
#     brewery_id = brewery_id.find('div', class_='content_title')
#     brewery_name = brewery_id.find('div', class_='article_teaser_body')
# result = {"Brewery_ID": brewery_df1['brewery_id'].to_list(),
#           "Brewery_Name": brewery_df1['brewery_name'].to_list()}
#     table_1.append(result)
# result
# review_df will have all the various reviews for each of the beer id's or beer_beerid
# review_df = brewery_df1[['beer_beerid', 'brewery_id','review_overall','review_aroma','review_appearance','review_palate','review_taste']].groupby('beer_beerid').mean().round(2)
# review_df
# brewery_df1


In [9]:
# import data to MongoDB
from pymongo import MongoClient
import json 
records= json.loads(brewery_df.to_json(orient='records'))

client = MongoClient()
client
client = MongoClient(host="localhost", port=27017)
db = client.BeerAdvocate_DB
collection = db.brewerydata

results = collection.insert_many(records)
results

In [10]:
# review_df will have all the various reviews for each of the beer id's or beer_beerid
review_df = beer_df[['beer_beerid', 'brewery_id','review_overall','review_aroma','review_appearance','review_palate','review_taste']].groupby('beer_beerid').mean().round(2)
review_df.reset_index()

,beer_beerid,brewery_id,review_overall,review_aroma,review_appearance,review_palate,review_taste
0,3,2.0,4.17,4.00,3.83,4.17,4.17
1,4,2.0,3.70,3.80,3.95,3.70,3.45
2,5,3.0,3.55,3.21,3.49,3.32,3.34
3,6,3.0,3.71,3.52,3.84,3.51,3.65
4,7,3.0,3.27,3.18,3.25,3.11,3.10
...,...,...,...,...,...,...,...
66050,77313,185.0,3.00,3.00,4.00,3.00,3.00
66051,77314,18604.0,3.50,3.50,4.00,3.00,4.00
66052,77315,27839.0,3.50,4.50,3.00,3.00,3.50
66053,77316,20330.0,3.50,3.50,3.50,3.50,3.50


In [11]:
# import data to MongoDB
from pymongo import MongoClient
import json 
records= json.loads(review_df.to_json(orient='records'))
# print (records)

client = MongoClient()
client
client = MongoClient(host="localhost", port=27017)
db = client.BeerAdvocate_DB
collection = db.reviewdata

results = collection.insert_many(records)
results

In [12]:
# beerid_df will have all the beer information such as name, style, and alcohol content for each beer_id
beerid_df = beer_df[['beer_beerid','beer_name','beer_style','beer_abv']].drop_duplicates('beer_beerid', keep='first').set_index('beer_beerid').sort_index()

In [13]:
# import data to MongoDB
from pymongo import MongoClient
import json 
records= json.loads(review_df.to_json(orient='records'))
# print (records)

client = MongoClient()
client
client = MongoClient(host="localhost", port=27017)
db = client.BeerAdvocate_DB
collection = db.beerdata

results = collection.insert_many(records)
results

In [ ]:
groupby('beerid_df','review_df' )

In [30]:
# Which brewry produces the strongest beers by ABV%? (the mean of ABV for each brewry has beed considered due to the lenght of dataset)
data = beer_df.groupby('brewery_name').beer_abv.mean()
strong = pd.DataFrame(data.reset_index())

strong.columns = ['brewery_name', 'beer_abv_mean']

strongest = strong[strong.beer_abv_mean == strong.beer_abv_mean.max()]
strongest

,brewery_name,beer_abv_mean
4667,Schorschbräu,19.228824


In [32]:
# If asked pick the Top 3 beer consider following factors:
# 1)review_overall, 
# 2) review_aroma, 
# 3) review_appearance 
# 4) review_palate, 
# 5) review_taste
# 6) beer_abv

reviews = beer_df.groupby('beer_beerid').agg({'review_overall': np.mean,
                                'review_aroma': np.mean,
                                'review_appearance': np.mean,
                                'review_palate': np.mean,     
                                'review_taste': np.mean,
                                'beer_abv': np.mean})

picks = pd.DataFrame(reviews.reset_index())

finaltop = picks.sort_values(['review_overall', 'review_aroma','review_appearance','review_palate','review_taste','beer_abv'], ascending=[False,False, False, False, False, False])
finaltop[:5]

,beer_beerid,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
46212,55451,5.0,5.0,5.0,5.0,5.0,13.0
64816,75956,5.0,5.0,5.0,5.0,5.0,11.0
40090,48325,5.0,5.0,5.0,5.0,5.0,10.5
1477,1734,5.0,5.0,5.0,5.0,5.0,10.0
1608,1890,5.0,5.0,5.0,5.0,5.0,10.0
